In [ ]:
# Carga de paquetes necesarios para graficar
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # Para leer archivos
import geopandas as gpd # Para hacer cosas geográficas
import seaborn as sns # Para hacer plots lindos
import networkx as nx # Construcción de la red en NetworkX
import scipy

np.seterr(divide='ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/ALC/')

In [ ]:
from template_funciones import *

# Preambulo

En esta sección cargamos los datos y los visualizamos. También construimos la matriz de adyacencia de la red de museos.

## Carga de datos de los museos

El listado de los museos, con el que se construye el [mapa](https://mapas.museosabiertos.org/museos/caba/), lo podemos encontrar [acá](https://github.com/MuseosAbiertos/Leaflet-museums-OpenStreetMap/blob/principal/data/export.geojson?short_path=bc357f3). También descargamos los barrios de CABA como complemento para los gráficos.

In [ ]:
# Leemos el archivo, retenemos aquellos museos que están en CABA, y descartamos aquellos que no tienen latitud y longitud
museos = gpd.read_file('https://raw.githubusercontent.com/MuseosAbiertos/Leaflet-museums-OpenStreetMap/refs/heads/principal/data/export.geojson')
barrios = gpd.read_file('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ministerio-de-educacion/barrios/barrios.geojson')

## Visualización

In [ ]:
# Armamos el gráfico para visualizar los museos
fig, ax = plt.subplots(figsize=(10, 10))
barrios.boundary.plot(color='gray',ax=ax)
museos.plot(ax=ax)

## Cálculo de la matriz de distancias

Ahora construimos la matriz de distancias entre todos los museos. Como la tierra es un [geoide](https://es.wikipedia.org/wiki/Geoide) (es decir que no es [plana](https://es.wikipedia.org/wiki/Terraplanismo)), el cálculo de distancias no es una operación obvia. Una opción es proyectar a un [sistema de coordenadas local](https://geopandas.org/en/stable/docs/user_guide/projections.html), de forma tal que las distancias euclideas se correspondan con las distancias en metros. En este notebook usamos [EPSG](https://en.wikipedia.org/wiki/EPSG_Geodetic_Parameter_Dataset) 22184.

In [ ]:
# En esta línea:
# Tomamos museos, lo convertimos al sistema de coordenadas de interés, extraemos su geometría (los puntos del mapa),
# calculamos sus distancias a los otros puntos de df, redondeamos (obteniendo distancia en metros), y lo convertimos a un array 2D de numpy
D = museos.to_crs("EPSG:22184").geometry.apply(lambda g: museos.to_crs("EPSG:22184").distance(g)).round().to_numpy()

### Matriz de adyacencia: construimos una matriz conectando a cada museo con los $m$ más cercanos

In [ ]:
m = 3 # Cantidad de links por nodo
A = construye_adyacencia(D,m)

## Construcción de la red en NetworkX (sólo para las visualizaciones)

In [ ]:
G = nx.from_numpy_array(A) # Construimos la red a partir de la matriz de adyacencia
# Construimos un layout a partir de las coordenadas geográficas
G_layout = {i:v for i,v in enumerate(zip(museos.to_crs("EPSG:22184").get_coordinates()['x'],museos.to_crs("EPSG:22184").get_coordinates()['y']))}

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15)) # Visualización de la red en el mapa
barrios.to_crs("EPSG:22184").boundary.plot(color='gray',ax=ax) # Graficamos Los barrios
nx.draw_networkx(G,G_layout,ax=ax) # Graficamos los museos

# Resolución del TP

Aquí empieza la aventura... ¡diviertanse y consulten lo que necesiten!

In [ ]:
def tres_museos_mas_centrales(C):
  """
  Dada la matriz C, devuelve los indices de los 3 museos mas centrales.

  Input:
    C: matriz de NxN

  Output:
    índices de los 3 museos mas centrales
  """
  # lista con cant. de elementos por columna.
  col_count = []
  for i in range(C.shape[1]):
      count = 0
      for elem in C[:,i]:
          count += 1 if elem != 0 else 0
      col_count.append(count)
  return np.argsort(col_count)[-3:]

## Punto 1:

Queremos ver que $\ M\vec{p} = \vec{b} \ \ $
con $\ \ M = \frac{N}{\alpha} \Bigl(I-\bigl(1-\alpha\bigl)C\Bigl) \ \ $
y $\ \ \vec{b} = \vec{1}$

\

$$
\text{Reemplazando:} \\
\\
M\vec{p} = \biggr[\frac{N}{\alpha} \Bigl(I-\bigl(1-\alpha\bigl)C\Bigl) \biggr] \vec{p} = \vec{1} = \vec{b} \\
\biggr[ \frac{N}{\alpha} \Bigl(I-\bigl(1-\alpha\bigl)C\Bigl) \biggr] \vec{p} = \vec{1} \\
\biggr[ \frac{N}{\alpha}I - \frac{N}{\alpha}\bigl(1-\alpha\bigl)C \biggr] \vec{p} = \vec{1} \\
\frac{N}{\alpha}I\vec{p} - \frac{N}{\alpha}\bigl(1-\alpha\bigl)C\vec{p} = \vec{1} \\
$$
\
$$
\text{Además, sabemos la ecuación de } \vec{p}: \\
\vec{p} = (1-\alpha)C\vec{p} + \frac{\alpha}{N}\vec{1}
$$
\
$$
\text{Reemplazando:} \\
\\
\frac{N}{\alpha}I \biggr[ (1-\alpha)C\vec{p} + \frac{\alpha}{N}\vec{1} \biggr] - \frac{N}{\alpha}\bigl(1-\alpha\bigl)C\vec{p} = \vec{1} \\
\frac{NI}{\alpha} (1-\alpha)C\vec{p} + \frac{N\alpha}{N\alpha}\vec{1}  - \frac{N}{\alpha}\bigl(1-\alpha\bigl)C\vec{p} = \vec{1} \\
\frac{N}{\alpha} I (1-\alpha)C\vec{p} + \vec{1}  - \frac{N}{\alpha}\bigl(1-\alpha\bigl)C\vec{p} = \vec{1} \\
$$
\
$$
\text{Como } IA = A: \\
\\
\frac{N}{\alpha} (1-\alpha)C\vec{p} + \vec{1}  - \frac{N}{\alpha}\bigl(1-\alpha\bigl)C\vec{p} = \vec{1} \\
$$
\
$$
\text{Cancelando nos queda finalmente:} \\
\vec{1} = \vec{1} \\
\text{Por ende probamos que es solución} \\
$$

<font color='orange'>Nico: Por que se reemplaza un solo p?</font>
<font color='orange'>Por otro lado, el enunciado buscaba que a partir de la ecuación 3 se despeje hasta llegar a Mp = b. Eso puede ser más facil. </font>


## Punto 2:


$$
M\vec{p} = \biggr[\frac{N}{\alpha} \Bigl(I-\bigl(1-\alpha\bigl)C\Bigl) \biggr] \vec{p} = \vec{b} \quad \text{tiene solucion unica}
\quad \Longleftrightarrow \quad
Nu(M)=\vec{0}
$$
\
$$
\text{Asumo que} \quad ∃ \ \vec{p}\neq\vec{0} \ / \ M\vec{p}=\vec{0} \\
\biggr[\frac{N}{\alpha} \Bigl(I-\bigl(1-\alpha\bigl)C\Bigl) \biggr] \vec{p} = \vec{0} \\
\Bigl(I-\bigl(1-\alpha\bigl)C\Bigl) \vec{p} = \vec{0} \\
I\vec{p}-\bigl(1-\alpha\bigl)C\ \vec{p} = \vec{0} \\
\vec{p} - \bigl(1-\alpha\bigl) C\vec{p} = \vec{0} \\
\vec{p} = \bigl(1-\alpha\bigl) C\vec{p} \\
\Vert \vec{p} \Vert_1 = \bigl(1-\alpha\bigl) \Vert C\vec{p} \Vert_1 \\
$$
<font color='orange'>Nico: El escalar sale con módulo, sabemos que 1 - alfa es positivo...</font>
\
$$
\text{Con} \quad \Vert C\vec{p} \Vert \le \Vert C \Vert \Vert \vec{p} \Vert \\
\Vert \vec{p} \Vert_1 \le \bigl(1-\alpha\bigl) \Vert C \Vert_1 \Vert \vec{p} \Vert_1 \\
$$

<font color='orange'>Nico: No hace falta asumir nada nuevo, ya se dijo arriba </font>
\
$$
\text{Asumo} \quad \Vert \vec{p} \Vert_1 \neq 0 \\
1 \le \bigl(1-\alpha\bigl) \Vert C \Vert_1 \\
$$

<font color='orange'>Nico: Lo que suma 1 en C son las columnas, ojo los indices de la sumatoria de abajo.</font>

\
$$
\text{Donde} \quad \Vert A \Vert_1 = \max_{1 \le j \le n} \sum_{i=1}^{n} \vert a_{ij} \vert \\
\text{Sin embargo se que} \quad \forall k \in [1,n] \quad \text{vale} \ \ \sum_{i=0}^{n} C_{ki} = 1 \\
\therefore \quad \max_{1 \le j \le n} \sum_{i=1}^{n} C_{ij} = 1 \\
\text{Como} \quad \forall i,j \quad C_{ij} > 0 \\
\max_{1 \le j \le n} \sum_{i=1}^{n} |C_{ij}| = 1 \\
\Vert C \Vert_1 = 1 \\
$$


\
$$
\text{Entonces} \quad 1 \le (1-\alpha) \quad \text{ABS!} \quad \text{ya que} \ \ \alpha \in (0,1) \\
\therefore \quad \nexists \ \ \vec{p} \neq \vec{0} \quad / \quad Nu(M) \neq \vec{0} \\
\text{Es decir que esta condición se cumple por construcción de la matriz} \ M
$$


<font color='greeb'>Nico: Hay algunos detalles, pero bien la demo!</font>


## Punto 3:

In [ ]:
# inciso a
%load_ext autoreload
%autoreload 2
from template_funciones import *


m = 3
alfa = 1/5

# calculo
A = construye_adyacencia(D, m)
C = calcula_matriz_C(A)
p_a = calcula_pagerank(A, alfa)

<font color='orange'>Nico: No están calculando correctamente la matriz K. El bug se arrastra a todo lo que sigue</font>

In [ ]:
# Visualizamos la red asignando un tamaño del nodo proporcional al page rank que le toca.
G = nx.from_numpy_array(C)
G_layout = {i:v for i,v in enumerate(zip(museos.to_crs("EPSG:22184").get_coordinates()['x'],museos.to_crs("EPSG:22184").get_coordinates()['y']))}

fig, ax = plt.subplots(figsize=(15, 15))
barrios.to_crs("EPSG:22184").boundary.plot(color='gray',ax=ax)
# escalamos para que se vean bien los puntos, son proporcionales a valores de p
nx.draw_networkx(G,G_layout,ax=ax, node_size = p_a * 10e3)


#### Inciso b

In [ ]:
m_list = [1, 3 , 5, 10]
alfa = 1/5


p_b_list = []
mas_centrales_b = []

for m in m_list:
  A = construye_adyacencia(D, m)
  p_b_list.append(calcula_pagerank(A, alfa))

  if mas_centrales_b == []:
    for museo in tres_museos_mas_centrales(C):
      mas_centrales_b.append(museo)


In [ ]:
"""
Identificamos 3 museos mas centrales por C y graficamos puntaje en funcion de
parámetro a variar.
"""

museos_mas_centrales = list(museos["name"].loc[mas_centrales_b])

p_individuales = []
for idx in mas_centrales_b:
  p = []
  for elem in p_b_list:
    p.append(elem[idx])
  p_individuales.append(p)

for i, p in enumerate(p_individuales):
  plt.scatter(m_list, p)
  l, = plt.plot(m_list, p)
  l.set_label(museos_mas_centrales[i],)
  plt.legend(fontsize=8)
plt.title("Puntaje PR en funcion de m")
plt.xlabel("m")
plt.ylabel("Puntaje")


Construimos mapa donde vamos cambiando el tamaño de los puntos en función del parámetro a cambiar.

In [ ]:
# obtenemos coordenadas
G_x = museos.to_crs("EPSG:22184").get_coordinates()['x']
G_y = museos.to_crs("EPSG:22184").get_coordinates()['y']
for i, p in enumerate(p_b_list):
  fig, ax = plt.subplots(figsize=(15, 15))
  barrios.to_crs("EPSG:22184").boundary.plot(color='gray',ax=ax)
  plt.scatter(G_x, G_y, s=p_b_list[i] * 10e3)
  plt.title(f"Mapa con m = {m_list[i]}")
  plt.show()


<font color='orange'>Nico: Deberían mostrar como cambia la estructura de la red,con los ejes. Podrían considerar plotear todos los mapas en subplot, uno al lado del otro. De esta forma sería más legible y comparable para su posterior discusión.</font>

#### Análisis

Podemos observar que al aumentar $m$, cambian las posiciones del ranking con respecto al valor inicial, por lo que no se observa estabilidad.

A medida que aumentamos el $m$, se observa que el puntaje de los museos que se encuentran en zonas más densas aumenta rápidamente. Sin embargo, cuando el $m$ excede la cantidad de museos que se encuentran en esa zona de mayor densidad, el puntaje se empieza a distribuir de forma más homogénea.

Notamos que hay museos que comienzan siendo relevantes, y luego no lo son. Esto se puede observar, por ejemplo, con el puntaje del Museo Nacional Ferroviario. Que con un valor pequeño de $m$, su puntaje es mayor, pero a medida que $m$ aumenta, su puntaje disminuye.


### Inciso c

In [ ]:
m = 5
alfa_list = [6/7, 4/5, 2/3, 1/2, 1/3, 1/5, 1/7]
alfa_list_str = ["6/7", "4/5", "2/3", "1/2", "1/3", "1/5", "1/7"]

p_c_list = []
mas_centrales_c = []

for alfa in alfa_list:
  A = construye_adyacencia(D, m)
  p_c_list.append(calcula_pagerank(A, alfa))
  for museo in tres_museos_mas_centrales(C):
    mas_centrales_c.append(museo)

# sacamos repetidos
mas_centrales_c = list(set(mas_centrales_c))

In [ ]:
"""
Identificamos 3 museos mas centrales por C y graficamos puntaje en funcion de
parámetro a variar.
"""

museos_mas_centrales = list(museos["name"].loc[mas_centrales_c])

p_individuales = []
for idx in mas_centrales_c:
  p = []
  for elem in p_c_list:
    p.append(elem[idx])
  p_individuales.append(p)

for i, p in enumerate(p_individuales):
  plt.scatter(alfa_list, p)
  l, = plt.plot(alfa_list, p)
  l.set_label(museos_mas_centrales[i],)
  plt.legend(fontsize=8)

plt.title("Puntaje PR en funcion de alpha")
plt.xlabel("alpha")
plt.ylabel("Puntaje")


<font color='orange'>Nico: Hay que identificar el top 3 para cada m (idem por alfa). Dicho de otra manera, queremos plotear la evolución del ranking de todo museo que alguna vez fue relevante al variar m (estuvo en el top3). Esto nos permite hacer una análisis un poco más profundo luego. </font>


Construimos mapa donde vamos cambiando el tamaño de los puntos en función del parámetro a cambiar.

In [ ]:
# obtenemos coordenadas
G_x = museos.to_crs("EPSG:22184").get_coordinates()['x']
G_y = museos.to_crs("EPSG:22184").get_coordinates()['y']
for i, p in enumerate(p_c_list):
  fig, ax = plt.subplots(figsize=(15, 15))
  barrios.to_crs("EPSG:22184").boundary.plot(color='gray',ax=ax)
  plt.scatter(G_x, G_y, s=p_c_list[i] * 10e3)
  plt.title(f"Mapa con alpha = {alfa_list_str[i]}")
  plt.show()


<font color='orange'>Nico: Misma aclaración que los otros para estos.</font>

#### Análisis

A medida que  $\alpha$ se acerca a 1, los puntajes de los museos parecen converger a un mismo punto con la particularidad de que las posiciones en el ranking se mantienen relativamente estables.

Esto sucede debido a que a medida que alpha se acerca a 1, la conexión entre museos pierde relevancia y la gana el factor aleatorio. Esto implica que mientras menor sea el valor de alpha más relevantes serán los museos con mayor cantidad de conexiones.


<font color='orange'>Nico: En realidad no importa solo la cantidad, si no la calidad de conexiones</font>


<font color='orange'>Nico: Las observacione habría que adaptarlas sin el bug y teniendo en cuenta el gráfico completo como mencionaba antes.</font>


<font color='orange'>Para profundizar la discusión, identifiquen casos concretos en el mapa que respalde lo que dicen en su discusión previa. Traten de explicar, en función de la intuición que se tiene del modelo, porqué tal museo mejora su ranking al incrementar m (o al revés). Prestar atención a la importancia de sus museos vecinos, las conexiones entrantes, etc. Apoyense en los gráficos de mapas y el de curvas. Puede ser importante mostrar las "flechas" del grafo para entender como se "transfiere" la relevancia de un museo a otro. </font>

<font color='orange'>También tengan presente la pregunta de estabilidad. Contestar esa pregunta con precisión involucraría definir estabilidad y definir algún tipo de métrica para "estabilidad". Una forma sería contar cantidad de permutaciones al variar m. No hace falta que lo hagan (aunque estaría bueno). Otra cosa que podría verse "a ojo" es en que intervalo de m o alfa el ranking es más inestable. Todo lo anterior son sugerencias para que puedan darle una vuelta más a esta pregunta </font>

## Punto 4:


Sea $w$ la cant. de visitantes totales de los museos.

En particular llamaremos $w_i$ a la cant. de visitantes después de $i$-recorridos. Análogamente llamaremos $v_j$ a la distribución de visitantes en el $j$-ésimo recorrido.

$$
\vec{w}_r = \vec{v}_0 + \vec{v}_1 + \vec{v}_2 + \dots + \vec{v}_{r-1} \\
\text{donde} \quad \vec{v}_r = C^r \vec{v}_0 \\
\vec{w}_r = \vec{v}_0 + C^1 \vec{v}_0 + C^2 \vec{v}_0 + \dots + C^{r-1} \vec{v}_0 \\
\vec{w}_r = \sum_{k=0}^{r-1} C^k \vec{v}_0 \\
\quad \text{Llamamos a } \vec{v}_0 = \vec{v} \text{  y  } \vec{w}_r = \vec{w} \\
{Entonces} \quad \vec{w} = \sum_{k=0}^{r-1} C^k \vec{v} \quad \text{y sabemos que} \quad \sum_{k=0}^{r-1} C^k = B  \\
\vec{w} = B \: \vec{v} \\
\text{Asumimos que B tiene inversa. Finalmente:} \\
\vec{v} = B^{-1} \vec{w}
$$

<font color='green'>Nico: Bien!</font>


## Punto 5:

In [ ]:
# cargamos v del archivo visitas.
import urllib.request

data = urllib.request.urlopen('https://raw.githubusercontent.com/z101ctrl/alc-metodos/refs/heads/main/visitas.txt')

visitas = []
for line in data:
  visitas.append(float(line.decode().strip()))

visitas = np.array(visitas)

In [ ]:
r = 3
C = calcula_matriz_C_continua(D)
B = calcula_B(C, r)
B_inv = inv(B)
v = B_inv @ visitas

cantidad_visitantes_iniciales = sum([abs(i) for i in v])
print("La norma 1 de v es:", cantidad_visitantes_iniciales)

La norma 1 de v es: 136604.99999999994


## Punto 6:

In [ ]:
condicion_1 = numero_de_condicion_1(B)
cota_error = condicion_1 * 0.05
print("La cota para el error es de", cota_error)
print("El número de condición de B es", condicion_1)

La cota para el error es de 0.25258855883325027
El número de condición de B es 5.051771176665005


<font color='orange'>Nico: Bien esta parte, agregaría un mínimo desarrollo (bien escrito en latex) que deje claro que es lo que se esta estimando en este punto</font>


# Extras

Para graficar la red con un conjunto de puntajes (como el Page Rank)

In [ ]:
factor_escala = 1e4 # Escalamos los nodos 10 mil veces para que sean bien visibles
fig, ax = plt.subplots(figsize=(10, 10)) # Visualización de la red en el mapa
barrios.to_crs("EPSG:22184").boundary.plot(color='gray',ax=ax) # Graficamos Los barrios
pr = np.random.uniform(0,1,museos.shape[0])# Este va a ser su score Page Rank. Ahora lo reemplazamos con un vector al azar
pr = pr/pr.sum() # Normalizamos para que sume 1
Nprincipales = 5 # Cantidad de principales
principales = np.argsort(pr)[-Nprincipales:] # Identificamos a los N principales
labels = {n: str(n) if i in principales else "" for i, n in enumerate(G.nodes)} # Nombres para esos nodos
nx.draw_networkx(G,G_layout,node_size = pr*factor_escala, ax=ax,with_labels=False) # Graficamos red
nx.draw_networkx_labels(G, G_layout, labels=labels, font_size=6, font_color="k") # Agregamos los nombres